# Importing Libraries:-
### 1. Pandas
### 2. NumPy
### 3. Regex

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import re

#### 
## Importing Data File:-
### 1. Initial inspection of data.
### 2. Understanding data.
#### 

In [ ]:
df = pd.read_csv("allocations.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df["Slot"].describe()

In [ ]:
df["Slot"].value_counts()

In [ ]:
df["Slot"].dtypes

### 
## Transformation of data:-
### 

In [ ]:
def convert(x):
    x = x.replace(" (IDC)","00")
    if x == ".":
        x = "16.3"
    return x

In [ ]:
convert("3 (IDC).1")

In [ ]:
df["Slot"] = df["Slot"].apply(convert)

#### 
### Day 16 means 2nd phase placements, Slot 1 is from morning to evening, slot 2 is from evening to late night, slot 3 is full day in phase 2. For IDC, days are multiplied by 100 to differentiate from general placements; like "Day 2' for IDC is represented as "Day 200" and so on.
#### 

In [ ]:
df[["Day","Slot No."]] = df["Slot"].str.split(".",expand = True)
df.head(10)

In [ ]:
df.tail()

In [ ]:
df.drop(columns = "Slot", inplace=True)

In [ ]:
df["Day"] = pd.to_numeric(df["Day"])

In [ ]:
df["Slot No."] = pd.to_numeric(df["Slot No."])

In [ ]:
df.head()

In [ ]:
df.drop(columns = "Job code", inplace=True)

In [ ]:
df.head()

### 
## Final inspection of data after the tranformation:-
### 

In [ ]:
df_BTech = df[df.Program == "B.Tech."]
df_BTech.head()

In [ ]:
df_MTech = df[df.Program == "M.Tech."]
df_MTech.head()

In [ ]:
df_Dual = df[df.Program.str.contains("Dual Degree")]
df_Dual.head()

In [ ]:
df.info()

#### 
##  Applying aggregation functions to group the data with similar features:-
### 1. Drawing insights from the data.
### 2. Storing insights in form of DataFrames.
#### 

## Slot wise analysis:-

In [ ]:
slot_data = df.groupby("Slot No.",as_index=0)["Roll"].count()
slot_data = slot_data.reset_index(drop = 1)
slot_data.rename({"Roll":"No. of Students"},axis=1,inplace=True)
slot_data                                                                                         # EXPORT AS SHEET

## Department and Program wise analysis:-

In [ ]:
Department_Stats_Breakdown = df.groupby(["Department","Program"], as_index = False)["Roll"].count().sort_values(["Roll","Program","Department"], ascending = [False,True,True])
Department_Stats_Breakdown.reset_index(drop = 1, inplace = True)
Department_Stats_Breakdown.head(15)

## Program wise analysis:-
### 1. Number count.
### 2. Percentage count.

In [ ]:
Program_Stats = df.groupby("Program",as_index = 0)["Roll"].count().sort_values("Roll",ascending = False)
Program_Stats.reset_index(drop = True, inplace= True)
Program_Stats.rename({"Roll":"No. of Students"},axis=1,inplace=True)
Program_Stats                                                                                    # EXPORT AS SHEET

In [ ]:
Program_Stats_Percentage = pd.DataFrame(round(df.value_counts("Program", normalize = True)*100,2))

In [ ]:
Program_Stats_Percentage.reset_index(inplace = True)
Program_Stats_Percentage.rename({0:"% of Students"},axis=1,inplace=True)
Program_Stats_Percentage                                                                       # EXPORT AS SHEET

## Company, Day and Slot wise mixed analysis:-

In [ ]:
company_stats = df.groupby(["Company","Day","Slot No."],as_index=False)["Roll"].count().sort_values(["Roll","Company"], ascending = [0,1])
company_stats.reset_index(drop = True, inplace=True)
company_stats.shape

In [ ]:
company_stats[company_stats["Company"].str.contains("Hp",flags=re.I, regex=True)]

In [ ]:
company_stats.rename({"Roll":"No. of Students"},axis=1,inplace=True)
company_stats.head(10)                                                                                 # EXPORT AS SHEET

In [ ]:
company_stats.info()

In [ ]:
company_stats["Company"].nunique()

In [ ]:
company_stats["Company"].count()

## Finding all the companies who recruited more than once:-

In [ ]:
repeated_companies = company_stats[company_stats.duplicated(subset="Company", keep=False)]
repeated_companies = repeated_companies.sort_values("Company")
repeated_companies.reset_index(drop=True,inplace=True)
repeated_companies.rename({"Roll":"No. of Students"},axis=1,inplace=True)
repeated_companies.head()                                                                            # EXPORT AS SHEET

In [ ]:
repeated_companies_total = repeated_companies.groupby(["Company"],as_index=False)["Roll"].sum()
repeated_companies_total = repeated_companies_stacked.sort_values(["Roll","Company"],ascending=[0,1])
repeated_companies_total.reset_index(drop=True,inplace=True)
repeated_companies_total.rename({"Roll":"No. of Students"},axis=1,inplace=True)
repeated_companies_total                                                                            # EXPORT AS SHEET

## Top recruiter Companies for the entire season:-

In [ ]:
top_companies = company_stats.groupby("Company",as_index=False)["Roll"].sum()
top_companies = top_companies.sort_values(["Roll","Company"],ascending=[0,1])
top_companies.reset_index(drop=True,inplace=True)
top_companies.rename({"Roll":"No. of Students"},axis=1,inplace=True)
top_companies.head(20)                                                                          # EXPORT AS SHEET

## Top branches:-

In [ ]:
def dept_splitter(x):
    splitted = x.split(", ")
    if len(splitted)==1:
        return x
    elif splitted[0]==splitted[1]:
        return splitted[0]
    else:
        return x

In [ ]:
top_branches = Department_Stats_Breakdown.copy()
top_branches["Department"] = top_branches["Department"].apply(dept_splitter)
top_branches = top_branches.rename({"Roll":"No. of Students"},axis=1)
top_branches.head(20)                                                                           # EXPORT AS SHEET

## Irrespective of Programs:-

In [ ]:
top_branches_all = top_branches.groupby("Department",as_index=False)["Roll"].sum()
top_branches_all = top_branches_all.sort_values(["Roll","Department"],ascending=[0,1])
top_branches_all.reset_index(drop = True, inplace = True)
top_branches_all.rename({"Roll":"No. of Students"},axis=1,inplace = True)                          # EXPORT AS SHEET
top_branches_all.head(10)

## Program wise:-

In [ ]:
top_branches_BTech = top_branches.loc[top_branches.Program == "B.Tech.",["Department","Roll"]]
top_branches_BTech.reset_index(drop = True, inplace=True)
top_branches_BTech.rename({"Roll":"No. of Students"},axis=1,inplace = True)
top_branches_BTech                                                                                # EXPORT AS SHEET

In [ ]:
top_branches_MTech = top_branches.loc[top_branches.Program == "M.Tech.",["Department","Roll"]]
top_branches_MTech.reset_index(drop = True, inplace=True)
top_branches_MTech.rename({"Roll":"No. of Students"},axis=1,inplace = True)
top_branches_MTech                                                                                # EXPORT AS SHEET

In [ ]:
top_branches_other = top_branches.loc[~top_branches.Program.isin(["M.Tech.","B.Tech."])]
top_branches_other = top_branches_other.rename({"Roll":"No. of Students"},axis=1)
top_branches_other.reset_index(drop=True,inplace=True)
top_branches_other.head()                                                                     # EXPORT AS SHEET

#### 
# Exporting DataFrames into multiple Excel sheets of a single file:-
#### 

In [ ]:
'''
with pd.ExcelWriter("Allocation_Analysis.xlsx") as w:
    df_BTech.to_excel(w,sheet_name="B.Tech Details",index=False,freeze_panes=(1,0))
    df_MTech.to_excel(w,sheet_name="M.Tech Details",index=False,freeze_panes=(1,0))
    df_Dual.to_excel(w,sheet_name="Dual Degree Details",index=False,freeze_panes=(1,0))
    slot_data.to_excel(w,sheet_name="Slot Analysis",index=False,freeze_panes=(1,0))
    Program_Stats.to_excel(w,sheet_name="Program_Stats",index=False,freeze_panes=(1,0))
    Program_Stats_Percentage.to_excel(w,sheet_name="Program_Stats_Percentage",index=False,freeze_panes=(1,0))
    company_stats.to_excel(w,sheet_name="Company_Stats",index=False,freeze_panes=(1,0))
    repeated_companies.to_excel(w,sheet_name="Repeating Companies",index=False,freeze_panes=(1,0))
    repeated_companies_total.to_excel(w,sheet_name="Best repeating Companies",index=False,freeze_panes=(1,0))
    top_companies.to_excel(w,sheet_name="Top Recruiters",index=False,freeze_panes=(1,0))
    top_branches.to_excel(w,sheet_name="Best Program & Department",index=False,freeze_panes=(1,0))
    top_branches_all.to_excel(w,sheet_name="Overall Best Department",index=False,freeze_panes=(1,0))
    top_branches_BTech.to_excel(w,sheet_name="Best B.Tech Department",index=False,freeze_panes=(1,0))
    top_branches_MTech.to_excel(w,sheet_name="Best M.Tech Department",index=False,freeze_panes=(1,0))
    top_branches_other.to_excel(w,sheet_name="Best_Dept_other_Programs",index=False,freeze_panes=(1,0))
'''